In [1]:
import pickle

In [2]:
import numpy as np
from datasets import load_from_disk
from torch.utils.data import DataLoader
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.manifold import TSNE

In [3]:
file_path = f"exp/2024.07.27/015329-default/dataset/generated_dataset_epoch_50.pkl"

# Open and read the pickle file
with open(file_path, 'rb') as f:
    generated_dataset = pickle.load(f)

In [4]:
len(generated_dataset)

6976

In [5]:
first_elements = [tup[1].numpy() for tup in generated_dataset]


In [6]:
first_elements = np.array(first_elements)

In [7]:
generated_ = first_elements[:,-51:-1]

In [8]:
generated_.shape

(6976, 50)

## RBC Dataset

In [9]:
dataset_rbc_path="data_interactions/RBCAgent1/model_RBCAgent1_timesteps_8760_rf_CombinedReward_phase_1.pkl"

In [10]:
dataset = load_from_disk(dataset_rbc_path)

In [11]:
from trajectory.utils.dataset_helper import DiscretizedDataset

In [12]:
datasets = DiscretizedDataset(dataset,discount = 0.99, seq_len = 2, strategy = "uniform")
   

Segmenting:   0%|          | 0/8759 [00:00<?, ?it/s]

Joining transitions:   0%|          | 0/1 [00:00<?, ?it/s]

Discount 0.99
(8759, 44)
(8759, 5)
(8759, 1)
(8759, 1)


In [13]:
datasets[8756][0]

array([ 6, 99, 75, 58, 53, 58, 54, 78, 75, 70, 73,  0,  0, 41,  0,  0,  0,
       62,  0, 68, 99,  3,  3, 99, 31,  0, 49, 46, 11,  0, 48, 28, 17,  0,
       49, 38,  5,  0, 48, 28, 18,  0, 49, 34, 20, 23, 22, 22, 22, 51, 97,
        6, 99, 85, 60, 51, 62, 56, 76, 75, 67, 66,  0,  0, 54,  0,  0,  0,
       51,  0, 68,  3,  3,  3,  3, 39,  0, 38, 54, 32,  0, 38, 47, 19,  0,
       38, 42, 17,  0, 37, 40, 43,  0, 38, 53, 24, 28, 27, 27, 27, 48],
      dtype=int64)

In [14]:
list_data_rbc = []
for i in datasets:
    list_data_rbc.append(i[0])

In [15]:
data_rbc = np.array(list_data_rbc)

In [16]:
data_rbc = data_rbc[:,:50]

In [17]:
data_rbc.shape

(8758, 50)

## concatenate

In [18]:
concatenate_ds = np.concatenate((data_rbc,generated_))

In [19]:
concatenate_ds.shape

(15734, 50)

In [20]:
tsne = TSNE(n_components=2, random_state=42)
features_tsne = tsne.fit_transform(concatenate_ds)

/home/ml-stud15/anaconda3/envs/stable3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/ml-stud15/anaconda3/envs/stable3/lib/python3.9/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [21]:
custom_cmap = ListedColormap(['green', 'black'])

In [22]:
num_samples = len(features_tsne)

In [23]:
labels = np.zeros(num_samples)
labels[8758:] = 1

In [24]:
plt.figure(figsize=(10, 8))

# Use a colormap and alpha for transparency
scatter = plt.scatter(features_tsne[:, 0], features_tsne[:, 1], c=labels, cmap=custom_cmap, alpha=0.6)

plt.title('t-SNE Visualization of The Dataset',fontsize=22)
plt.xlabel('t-SNE Component 1',fontsize=20)
plt.ylabel('t-SNE Component 2',fontsize=20)
plt.legend(handles=scatter.legend_elements()[0], labels=['Original Dataset', 'Generated Dataset'],fontsize=16)
plt.ylim(-100,100)
plt.xlim(-100,100)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.savefig("figs_generated/final_epoch")
plt.close()
#plt.show()